# **1. Data Loading and Preprocessing**

## **Introduction**
The initial phase of this study involves **data loading and preprocessing**, which lays the foundation for effective analysis and machine learning modeling. The dataset under consideration contains **568,454 user-generated reviews**, capturing textual feedback along with metadata such as numerical ratings, timestamps, and helpfulness scores. Given the sheer volume and diversity of the dataset, meticulous preprocessing is critical to ensure reliability and accuracy in downstream tasks.

## **Objectives of this Section**
- **Efficiently load the dataset** while minimizing memory usage.
- **Verify dataset integrity** by inspecting shape and column structure.
- **Handle missing values and duplicates** to prevent distortions in analysis.
- **Select only relevant features** that contribute to the classification task.
- **Engineer a new feature (`Combined_Text`)** by merging short (`Summary`) and long (`Text`) reviews.
- **Transform numerical ratings (`Score`) into categorical satisfaction levels (`User_Satisfaction`)** for supervised classification.

## **Dataset Structure (Before Processing)**
Upon loading the dataset, we retrieve the following properties:

In [5]:
import pandas as pd

# Load the data
data = pd.read_csv('Reviews.csv', encoding='ISO-8859-1', low_memory=False)

# Display dataset shape to confirm full loading
print(f"Dataset shape: {data.shape}")

# Show column names to verify
print(f"Columns: {data.columns}")


Dataset shape: (568454, 10)
Columns: Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')


The dataset comprises **568,454 rows** and **10 columns**, capturing a mix of numerical, categorical, and textual data. 

## **Dataset Column Descriptions**

The dataset consists of user-generated reviews, capturing essential details about products, users, ratings, and feedback. Each row represents an individual review, accompanied by metadata that provides context for the evaluation.

---

## **Column Descriptions**

### **1. Id**
- **Definition**: A unique identifier assigned to each row in the dataset.
- **Type**: Integer

### **2. ProductId**
- **Definition**: A unique alphanumeric identifier associated with each product.
- **Type**: String

### **3. UserId**
- **Definition**: A unique identifier assigned to each user who submitted a review.
- **Type**: String

### **4. ProfileName**
- **Definition**: The display name of the user who submitted the review.
- **Type**: String

### **5. HelpfulnessNumerator**
- **Definition**: The number of users who found the review helpful.
- **Type**: Integer

### **6. HelpfulnessDenominator**
- **Definition**: The total number of users who provided feedback on whether a review was helpful.
- **Type**: Integer

### **7. Score**
- **Definition**: The rating provided by the user, ranging from 1 to 5.
- **Type**: Integer (1-5)
- **Purpose**: Represents the user's sentiment about the product.
  - **5** → Highly Satisfied
  - **4** → Satisfied
  - **3** → Neutral
  - **2** → Not Satisfied
  - **1** → Very Bad

### **8. Time**
- **Definition**: A Unix timestamp representing the date and time when the review was posted.
- **Type**: Integer

### **9. Summary**
- **Definition**: A brief, high-level summary of the review provided by the user.
- **Type**: String

### **10. Text**
- **Definition**: The full-length detailed review written by the user.
- **Type**: String
---

## **Data Cleaning Steps**
1. **Handling Missing Values**:
   - Since the `Score` column is central to our classification task, all rows with `NaN` values in `Score` are removed.
   - Missing values in textual fields (`Summary` and `Text`) are replaced with empty strings (`""`) to ensure consistency in text processing.

2. **Removing Duplicates**:
   - Identical reviews can distort classification models by **artificially inflating the occurrence of certain labels**. Therefore, we eliminate duplicate rows.

3. **Feature Selection**:
   - To reduce computational complexity and enhance model interpretability, we retain only the most pertinent columns:
     - `HelpfulnessNumerator`: Number of users who found the review helpful.
     - `HelpfulnessDenominator`: Total number of users who rated helpfulness.
     - `Score`: The numerical rating given by the reviewer (1 to 5).
     - `Summary`: A brief overview of the review.
     - `Text`: The full review content.

4. **Feature Engineering (`Combined_Text`)**:
   - The `Summary` column contains **concise expressions of user sentiment**, whereas the `Text` column provides **detailed qualitative feedback**.
   - To **capture both short- and long-form sentiment**, we concatenate them into a new column:  
     ```
     Combined_Text = Summary + " " + Text
     ```

## **Transforming the `Score` Column**
The numerical `Score` column is mapped to categorical `User_Satisfaction` levels as follows:

In [6]:
# Remove rows where 'Score' is empty
data = data[data["Score"].notna()]
data = data.drop_duplicates()


# Select only the relevant columns
selected_columns = [
    "HelpfulnessNumerator", 
    "HelpfulnessDenominator", 
    "Score", 
    "Time", 
    "Summary", 
    "Text"
]

# Keep only the selected columns
data = data[selected_columns]

data["Score"] = pd.to_numeric(data["Score"], errors='coerce')
data["Combined_Text"] = data["Summary"] + " " + data["Text"]

# Define function to categorize User_Satisfaction
def classify_satisfaction(rating):
    if rating == 5:
        return "Highly Satisfied"
    elif rating == 4:
        return "Satisfied"
    elif rating == 3:
        return "Neutral"
    elif rating == 2:
        return "Not Satisfied"
    elif rating <= 1:
        return "Very Bad"
    else:
        return None  # Handle missing values gracefully

# Apply function to create new column
data["User_Satisfaction"] = data["Score"].apply(classify_satisfaction)

data

,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Combined_Text,User_Satisfaction
0,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,Good Quality Dog Food I have bought several of...,Highly Satisfied
1,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised Product arrived labeled as J...,Very Bad
2,1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,"""Delight"" says it all This is a confection tha...",Satisfied
3,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,Cough Medicine If you are looking for the secr...,Not Satisfied
4,0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...,Great taffy Great taffy at a great price. The...,Highly Satisfied
...,...,...,...,...,...,...,...,...
568449,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...,Will not do without Great for sesame chicken.....,Highly Satisfied
568450,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...,disappointed I'm disappointed with the flavor....,Not Satisfied
568451,2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o...",Perfect for our maltipoo These stars are small...,Highly Satisfied
568452,1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...,Favorite Training and reward treat These are t...,Highly Satisfied


This transformation enables the classification task to shift from a **numeric regression problem to a categorical classification problem**.

## **Class Distribution (After Processing)**


In [7]:
user_satisfaction_counts = data["User_Satisfaction"].value_counts()

# Display the counts
print(user_satisfaction_counts)

User_Satisfaction
Highly Satisfied    363122
Satisfied            80655
Very Bad             52268
Neutral              42640
Not Satisfied        29769
Name: count, dtype: int64


# **2. Data Splitting for Machine Learning**

## **Why Splitting the Data?**
Before training a classification model, it is crucial to **partition the dataset into training and testing subsets**. This ensures:
- **Generalizability**: The model is evaluated on unseen data.
- **Avoiding Overfitting**: Prevents the model from memorizing patterns in training data.
- **Measuring Model Performance**: Provides an unbiased estimate of accuracy.

## **Data Preparation Steps**
1. **Label Encoding**:
   - Since `User_Satisfaction` is a categorical variable, it is transformed into **numerical labels** via `LabelEncoder()`.
   - Example:
     ```
     "Highly Satisfied" → 0
     "Satisfied" → 1
     "Neutral" → 2
     "Not Satisfied" → 3
     "Very Bad" → 4
     ```

2. **Train-Test Split**:
   - We perform an **80-20 split**, where **80%** of the data is used for training and **20%** for evaluation.
   - **Stratified Sampling** ensures that the **class distribution remains proportional** in both training and test sets.

## **Resulting Data Structure**


In [8]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset

# Select the required columns
selected_columns = [
    "HelpfulnessNumerator", 
    "HelpfulnessDenominator", 
    "Summary",  
    "Text", 
    "User_Satisfaction"
]
data = data[selected_columns]
# Fill missing values with an empty string
data[["Summary", "Text"]] = data[["Summary", "Text"]].fillna("")

# Combine 'Summary' + 'Text' for text processing
data["Combined_Text"] = data["Summary"] + " " + data["Text"]

# Encode 'User_Satisfaction' as numerical labels
label_encoder = LabelEncoder()
data["User_Satisfaction"] = label_encoder.fit_transform(data["User_Satisfaction"])

# Train-Test Split (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    data[["HelpfulnessNumerator", "HelpfulnessDenominator", "Combined_Text"]],
    data["User_Satisfaction"],
    test_size=0.2,
    random_state=42,
    stratify=data["User_Satisfaction"]
)

# Display Data Splitting Results
print(f"Training Set: {X_train.shape}, Test Set: {X_test.shape}")


C:\Users\saisa\AppData\Local\Temp\ipykernel_24188\1625214675.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[["Summary", "Text"]] = data[["Summary", "Text"]].fillna("")
C:\Users\saisa\AppData\Local\Temp\ipykernel_24188\1625214675.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Combined_Text"] = data["Summary"] + " " + data["Text"]
C:\Users\saisa\AppData\Local\Temp\ipykernel_24188\1625214675.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Training Set: (454763, 3), Test Set: (113691, 3)


The final dataset consists of:
- `HelpfulnessNumerator`
- `HelpfulnessDenominator`
- `Combined_Text`

With target variable as `User_Satisfaction`